In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Функция ```prepare_mask``` сдвигает матрицу перехода ```mask``` на ```shift``` элементов вправо относительно её главного элемента ```pivot``` с учётом вектора смещения ```wind_direction```

In [ ]:
def prepare_mask(mask, shift, n):
    if shift < len(mask) // 2:
        mask_ = mask[(len(mask) // 2 - shift):]
    elif n - shift - 1 < len(mask) // 2:
        mask_ = mask[:(len(mask) // 2 + (n - shift))]
    else:
        mask_ = mask
    leftpad = max(0, shift - len(mask) // 2)
    rightpad = max(0, n - len(mask_) - leftpad)
    mask_ = np.pad(mask_, (leftpad, rightpad), 'constant')
    return mask_

##### С использованием современных технологий, центр управления способен отслеживать синоптические данные в режиме реального времени. С использованием информации о параметрах ветра в районе проведения поисков становится возможным предсказать вероятности нахождения оторвавшейся льдины в каждой из областей. В данном случае в качестве упрощения в модели берётся фиксированный квант времени, за который вероятности перемещения льдины заранее известны

Функция ```iteration``` производит одну итерацию применения матрицы перехода ```mask``` к каждому элементу массива вероятностей ```prob```

In [ ]:
def iteration(prob, mask):
    n = len(prob)
    prob_ = np.zeros((n))
    for i in range(0, n):
        prob_ += prob[i] * prepare_mask(mask, i, n)
    return prob_

##### В моделе предусмотрено введение в систему дополнительной информации с целью уточнения и корректировки уже просчитанных вероятностей. Предоставляется интерфейс ввода вероятностей нахождения льдины в каждой из областей. Это мотивировано спецификой работы нейронных сетей распознавания образов - на выходе модели мы всегда получаем вероятность обнаружения объекта. Таким образом, на практике можно использовать любй доступный беспилотный летательный аппарат с возможностью анализа входного видеопотока с наружней камеры для определения вероятности нахождения льдины с людьми в той области, которая в данный момент просматривается БЛА. Данный подход также мотивирован неидеальностью условий, в которых человек производит наблюдения. Даже в случае задействования спасательного вертолета с наблюдателем, погодные условия (туман, осадки...), высота, острота зрения, посторонние объекты на льдинах (тёмные камни, животные...) и прочие факторы могут сказаться на уверенности спасателя в обнаружении нужной льдины с людьми

Функция ```input_information``` заполняет массив ```h``` априорными вероятностями гипотез

In [ ]:
def input_information(n):
    h = [None] * n
    return h

Функция ```clarify``` использует формулу Байеса для вычисления апостериорных вероятностей для каждого из элементов массива вероятностей ```prob``` используя априорные вероятности ```h```

In [ ]:
def clarify(prob, h):
    prob_ = prob
    n = len(prob)
    for i in range(n):
        if h[i] is not None:
            prob_[i] *= h[i]
    prob_ /= np.sum(prob_)
    return prob_

#### Визуальное отображение результатов вычислений важно для принятия решений спасательным центром ввиду абстрагирования данной модели от различных комплексных факторов, таких как риск проведения спасательной операции, особенности географии местности, оперативные данные и пр.

Функция ```plot_probability``` графически отображает массив вероятностей ```prob```

In [ ]:
def plot_probability(prob):
    n = len(prob)
    plt.xlim(0, n - 1)
    plt.ylim(0, 1)
    plt.plot(prob)

In [ ]:
n = 100
prob = np.zeros((n))
prob[50] = 1
mask = np.array([0.25, 0.5, 0.25]) # must be odd

In [ ]:
prob = iteration(prob, mask)
prob = clarify(prob, input_information(n))

In [ ]:
plot_probability(prob)